#  NOAA Janus taxa list
## 101-210 taxa

Create list of taxa for NOAA Janus files. Compare NOAA taxa with the taxa that the PIs have already approved in order create a list of unapproved taxa. Add PBDB data to unapproved taxa.

In [1]:
import sys
import csv
import glob
import os
import requests
import re
import shutil

sys.path.append('../../')

from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR


import pandas as pd
import numpy as np

import db 
import scripts.normalize_taxa as nt
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb 


In [2]:
# base_directory = 'cleaned_data'
date='2022-11-15'

lims_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_{date}.csv'
lims_crosswalk_path = OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_crosswalk_{date}.csv'
lims_genus_path = OUTPUT_DIR/'taxa'/'LIMS'/f'genera_pbdb_{date}.csv'


date='2022-11-18'

metadata_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_janus_iodp_files.csv'
noaa_101_210_taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_101_210_{date}.csv'
unapproved_taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_unapproved_101_210_{date}.csv'
species_path = OUTPUT_DIR/'taxa'/'NOAA'/f'species_101_210_{date}.csv'
genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_101_210_{date}.csv'
higher_path = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_101_210_{date}.csv'


metadata_1_96_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_dsdp_files.csv'
noaa_1_96_taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_1_96_{date}.csv'

PI_noaa_1_96_taxa_path =  RAW_DATA_DIR/'PI_processed_files'/'NOAA_taxa_lists_taxa_list_2022-11-15.csv'



## Create taxa list

In [3]:
metadata = pd.read_csv(metadata_path)
metadata = metadata[metadata['type'] == 'taxa']
log_df(metadata)

(2045, 5)


,path,type,expedition,site,taxon_group
0,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,benthic_forams
1,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,benthic_forams
2,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,nannofossils
3,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,nannofossils
4,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,planktic_forams


In [4]:
taxa_df = nt.create_noaa_2_taxa_crosswalk_df(metadata, CLEAN_DATA_DIR)
log_df(taxa_df)
# 13066

(13066, 10)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor
9761,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,,?,Labyrinthodinium,<NA>,sp.,<NA>,1,<NA>
5657,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,,?,Maduradinium,<NA>,sp.,<NA>,<NA>,<NA>
2305,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,,?,Pyxidiella,<NA>,sp.,<NA>,1,<NA>
1302,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,,<NA>,Aandalusiella,<NA>,ivoirensis,<NA>,<NA>,<NA>
8448,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,,<NA>,Abratopdinium,<NA>,cardioforme,<NA>,<NA>,<NA>


In [5]:
taxa_df.to_csv(noaa_101_210_taxa_path, index=False)

## create noaa 1 taxa

In [5]:
metadata_1 = pd.read_csv(metadata_1_96_path)
metadata_1 = metadata_1[metadata_1['type'] == 'taxa']
log_df(metadata_1)

(2093, 5)


,path,type,taxon_group,expedition,site
0,NOAA/DSDP_core_data/61/462/radiolar.csv,taxa,radiolarians,61,462
2,NOAA/DSDP_core_data/61/462/b_forams.csv,taxa,benthic_forams,61,462
3,NOAA/DSDP_core_data/61/462/p_forams.csv,taxa,planktic_forams,61,462
5,NOAA/DSDP_core_data/61/462/nannos.csv,taxa,nannofossils,61,462
7,NOAA/DSDP_core_data/61/462A/radiolar.csv,taxa,radiolarians,61,462A


In [6]:
taxa_1_df = nt.create_noaa_1_taxa_crosswalk_df(metadata_1, CLEAN_DATA_DIR)
log_df(taxa_1_df)
# 9933

(9933, 10)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor
2076,Ebridians and Actinicidians,Actiniscus elongatus (q),? Actiniscus elongatus,?,Actiniscus,<NA>,elongatus,<NA>,<NA>,<NA>
471,Ebridians and Actinicidians,Actiniscus laciniatus (q),? Actiniscus laciniatus,?,Actiniscus,<NA>,laciniatus,<NA>,<NA>,<NA>
2752,Ebridians and Actinicidians,Actiniscus pentasterias,,<NA>,Actiniscus,<NA>,pentasterias,<NA>,<NA>,<NA>
5794,Ebridians and Actinicidians,Actiniscus sp.,,<NA>,Actiniscus,<NA>,sp.,<NA>,<NA>,<NA>
1261,Ebridians and Actinicidians,Actiniscus squamosus,,<NA>,Actiniscus,<NA>,squamosus,<NA>,<NA>,<NA>


In [7]:
taxa_1_df.to_csv(noaa_1_96_taxa_path, index=False)

## Created taxa list with unapproved NOAA taxa

In [10]:
noaa_2_taxa_df = pd.read_csv(noaa_101_210_taxa_path)
log_df(noaa_2_taxa_df)

# 13066

(13066, 10)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN


In [11]:
noaa_1_taxa_df = pd.read_csv(noaa_1_96_taxa_path, usecols=['taxon_group', 'verbatim_name'])
log_df(noaa_1_taxa_df)

# 9933

(9933, 2)


,taxon_group,verbatim_name
0,Ebridians and Actinicidians,Actiniscus elongatus (q)
1,Ebridians and Actinicidians,Actiniscus laciniatus (q)
2,Ebridians and Actinicidians,Actiniscus pentasterias
3,Ebridians and Actinicidians,Actiniscus sp.
4,Ebridians and Actinicidians,Actiniscus squamosus


In [12]:
lims_taxa_df = pd.read_csv(lims_crosswalk_path, usecols=['taxon_group', 'verbatim_name'])

log_df(lims_taxa_df)
# 5380

(5380, 2)


,taxon_group,verbatim_name
0,benthic_forams,Euuvigerina miozea (group) >100 m
1,benthic_forams,Euuvigerina rodleyi (group) >50 m
2,benthic_forams,Others
3,benthic_forams,Pleurostomellids comment
4,benthic_forams,Ostracoda spp.


get taxa that PIs have looked at

In [13]:
approved_taxa_df = pd.concat([noaa_1_taxa_df, lims_taxa_df])
approved_taxa_df.drop_duplicates(inplace=True)

log_df(approved_taxa_df)
# 13588

(13588, 2)


,taxon_group,verbatim_name
0,Ebridians and Actinicidians,Actiniscus elongatus (q)
1,Ebridians and Actinicidians,Actiniscus laciniatus (q)
2,Ebridians and Actinicidians,Actiniscus pentasterias
3,Ebridians and Actinicidians,Actiniscus sp.
4,Ebridians and Actinicidians,Actiniscus squamosus


select NOAA taxa that needs approval

In [14]:
merged_df = noaa_2_taxa_df.merge(approved_taxa_df,  
                     on=['verbatim_name', 'taxon_group' ], 
                     how='left',
                     indicator='_merge_type')

merged_df = merged_df.drop_duplicates()

log_df(merged_df)
# 13066 

(13066, 11)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,_merge_type
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,left_only
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,left_only
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,left_only
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,left_only
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,left_only


Select unapproved NOAA taxa. When _merged is both, it means the taxa has been approved. When _merged is left_only, it means the taxa has not been approved.

In [15]:
unapproved_taxa_df = merged_df[merged_df['_merge_type'] == 'left_only'].copy()

del unapproved_taxa_df['_merge_type']

log_df(unapproved_taxa_df)

# (9024, 9)

(9024, 10)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN


In [16]:
unapproved_taxa_df.to_csv(unapproved_taxa_path, index=False)

## create species csv

Look up the genus for unapproved taxa in PBDB

In [73]:
def add_genus_species(taxa_df):
    taxa_df.loc[~taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True).fillna(False), 'genus species']=taxa_df['genus name'] + ' ' + taxa_df['species name']
    # taxa_df.loc[taxa_df['species name'].isna(), 'genus species']=taxa_df['genus name'] 
    # taxa_df.loc[taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True).fillna(False), 'genus species']=taxa_df['genus name']
    taxa_df['genus species'] = taxa_df['genus species'].str.replace('(?)', '', regex=False)
    taxa_df['genus species'] = taxa_df['genus species'].str.replace('?', '', regex=False)

    taxa_df['genus species'] = taxa_df['genus species'].str.strip()

In [74]:
df = pd.read_csv(unapproved_taxa_path)
add_genus_species(df)
log_df(df)

(9024, 25)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,Any taxon above genus,genus species
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,321606.0,Gonyaulacales,321578.0,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,277919.0,Peridiniales,321578.0,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,277919.0,Peridiniales,321578.0,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aandalusiella ivoirensis
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abratopdinium cardioforme


In [75]:
species_df = df[df['genus species'].notna()].copy()[['taxon_group', 'genus species']]
species_df.drop_duplicates(inplace=True)
species_df['check'] = False

log_df(species_df)
# 5525

(5525, 3)


,taxon_group,genus species,check
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,False
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,False
5,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium kerguelense,False
6,Dinoflagellates/Acritarchs/Prasinophytes,Acanthaulax granulata,False
7,Dinoflagellates/Acritarchs/Prasinophytes,Acanthaulax wilsonii,False


In [90]:
pbdb.fetch_pdbd_data(species_df, 'genus species')

In [91]:
species_df[species_df['check'] == False].shape

(0, 20)

In [94]:
# del species_df['check']


In [95]:
species_df.to_csv(species_path, index=False)

## add species pbdb info to unapproved taxa 

In [3]:
def add_genus_species(taxa_df):
    taxa_df.loc[~taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True).fillna(False), 'genus species']=taxa_df['genus name'] + ' ' + taxa_df['species name']
    # taxa_df.loc[taxa_df['species name'].isna(), 'genus species']=taxa_df['genus name'] 
    # taxa_df.loc[taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True).fillna(False), 'genus species']=taxa_df['genus name']
    taxa_df['genus species'] = taxa_df['genus species'].str.replace('(?)', '', regex=False)
    taxa_df['genus species'] = taxa_df['genus species'].str.replace('?', '', regex=False)

    taxa_df['genus species'] = taxa_df['genus species'].str.strip()

In [4]:
species_df = pd.read_csv(species_path, dtype=str)

log_df(species_df)
# 5525

(5525, 19)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
species_df = species_df[species_df['pbdb_taxon_name'].notna() & (species_df['pbdb_taxon_rank'] == 'species')]
log_df(species_df)
# 960

(960, 19)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
5,Dinoflagellates/Acritarchs/Prasinophytes,Achilleodinium bianii,323992,Achilleodinium bianii,species,323991,Achilleodinium,321603,Gonyaulacaceae,321606,Gonyaulacales,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
15,Dinoflagellates/Acritarchs/Prasinophytes,Achomosphaera ramulifera,277049,Achomosphaera ramulifera,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,Gonyaulacales,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
16,Dinoflagellates/Acritarchs/Prasinophytes,Achomosphaera sagena,323552,Achomosphaera sagena,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,Gonyaulacales,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
17,Dinoflagellates/Acritarchs/Prasinophytes,Achomosphaera triangulata,323940,Achomosphaera triangulata,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,Gonyaulacales,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
18,Dinoflagellates/Acritarchs/Prasinophytes,Achomosphaera verdieri,323553,Achomosphaera verdieri,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,Gonyaulacales,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
unapproved_df = pd.read_csv(unapproved_taxa_path, dtype=str)
add_genus_species(unapproved_df)
log_df(unapproved_df)
# (9024, 11)


(9024, 11)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,genus species
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,Aandalusiella ivoirensis
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,Abratopdinium cardioforme


In [7]:
pbdb.add_pbdb_data(unapproved_df, species_df, 'genus species')

In [8]:
diff = set(species_df.columns) - set(unapproved_df.columns)
diff

set()

In [9]:
log_df(unapproved_df)
# (9024, 28)

(9024, 28)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,genus_taxon_id,genus_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
unapproved_df.to_csv(unapproved_taxa_path, index=False)

## create genus csv

In [26]:
lims_genus_df = pd.read_csv(lims_genus_path, dtype=str)
log_df(lims_genus_df)

(1026, 15)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,benthic_forams,Euuvigerina,1408,Euuvigerina,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,benthic_forams,Nodosaria,1952,Nodosaria,genus,82197,Nodosariidae,429322,Nodosariida,428875,Nodosariata,288974,Foraminifera,212476,Rhizaria
2,benthic_forams,Cibicides,1107,Cibicides,genus,82208,Cibicididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,benthic_forams,Brizalina,1017,Brizalina,genus,112279,Bolivinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,planktic_forams,Candeina,1053,Candeina,genus,422277,Candeinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


In [27]:
df = pd.read_csv(unapproved_taxa_path)
log_df(df)

(9024, 28)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,genus_taxon_id,genus_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
genus_df = df[df['genus name'].notna()].copy()[['taxon_group', 'genus name']]
genus_df.drop_duplicates(inplace=True)
genus_df['check'] = False

log_df(genus_df)

(2545, 3)


,taxon_group,genus name,check
0,Dinoflagellates/Acritarchs/Prasinophytes,Labyrinthodinium,False
1,Dinoflagellates/Acritarchs/Prasinophytes,Maduradinium,False
2,Dinoflagellates/Acritarchs/Prasinophytes,Pyxidiella,False
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella,False
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium,False


In [29]:
genus_df = genus_df.merge(lims_genus_df, how='left')
genus_df.loc[genus_df['pbdb_taxon_id'].notna(), 'check'] = True

log_df(genus_df)

(2545, 16)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,Labyrinthodinium,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,Maduradinium,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,Pyxidiella,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
pbdb.fetch_pdbd_data(genus_df, 'genus name')

Index(['taxon_group', 'genus name', 'check', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name',
       'unranked clade_taxon_id', 'unranked clade_taxon_name',
       'subclass_taxon_id', 'subclass_taxon_name', 'genus_taxon_id',
       'genus_taxon_name'],
      dtype='object')
800 1000 1350 1750 1800 1850 2100 2150 2200 2250 2300 2350 2400 2450 2500 

In [106]:
genus_df[genus_df['check'] == False].shape

(0, 22)

In [107]:
log_df(genus_df)

(2545, 22)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,Labyrinthodinium,True,443826,Labyrinthodinium,genus,NaN,NaN,321606,Gonyaulacales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,Maduradinium,True,325673,Maduradinium,genus,277915,Peridiniaceae,277919,Peridiniales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,Pyxidiella,True,336773,Pyxidiella,genus,277915,Peridiniaceae,277919,Peridiniales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
del genus_df['check']

In [110]:
genus_df.to_csv(genus_path, index=False)

## add genus pbdb info to unapproved taxa 

In [11]:
genus_df = pd.read_csv(genus_path, dtype=str)
log_df(genus_df)
# (2545, 21)

(2545, 21)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,Labyrinthodinium,443826,Labyrinthodinium,genus,NaN,NaN,321606,Gonyaulacales,321578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,Maduradinium,325673,Maduradinium,genus,277915,Peridiniaceae,277919,Peridiniales,321578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,Pyxidiella,336773,Pyxidiella,genus,277915,Peridiniaceae,277919,Peridiniales,321578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
unapproved_df = pd.read_csv(unapproved_taxa_path, dtype=str)

log_df(unapproved_df)
#  (9024, 28)


(9024, 28)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,genus_taxon_id,genus_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pbdb.add_pbdb_data(unapproved_df, genus_df, 'genus name')

In [14]:
diff = set(genus_df.columns) - set(unapproved_df.columns)
diff

set()

In [15]:
log_df(unapproved_df)
# (9024, 30)

(9024, 30)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,genus_taxon_id,genus_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,subclass_taxon_id,subclass_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
unapproved_df.to_csv(unapproved_taxa_path, index=False)

## update columns in taxa list

In [29]:
unapproved_df = pd.read_csv(unapproved_taxa_path, dtype=str)

log_df(unapproved_df)
#  (9024, 30)

(9024, 30)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,genus_taxon_id,genus_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,subclass_taxon_id,subclass_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
PI_df = pd.read_csv(PI_noaa_1_96_taxa_path, dtype=str)

In [31]:
set([col.replace('.1', '') for col in PI_df.columns]) - set(unapproved_df.columns)

{'Any taxon above genus',
 'Any taxon above genus modifier',
 'Comment',
 'Corrections to pbdb_taxon',
 'Corrections to pbdb_taxon_rank',
 'Notes (change to Internal only notes?)',
 'comments',
 'subgenera modifier',
 'subgenera name',
 'superfamily_taxon_id',
 'superfamily_taxon_name'}

In [32]:
unapproved_df['Any taxon above genus'] = pd.NA
unapproved_df['Any taxon above genus modifier'] = pd.NA
unapproved_df['Comment'] = pd.NA
unapproved_df['Corrections to pbdb_taxon'] = pd.NA
unapproved_df['Notes (change to Internal only notes?)'] = pd.NA
unapproved_df['comments'] = pd.NA
unapproved_df['subgenera modifier'] = pd.NA
unapproved_df['subgenera name'] = pd.NA
unapproved_df['species_taxon_id'] = pd.NA
unapproved_df['species_taxon_name'] = pd.NA

del unapproved_df['genus species']

In [33]:
len(unapproved_df.columns)
# 39

39

In [34]:
for rank in ['species', 'genus', 'family', 'order', 'class', 'phylum']:
    unapproved_df.loc[unapproved_df['pbdb_taxon_rank'] == rank, f'{rank}_taxon_name' ] = unapproved_df['pbdb_taxon_name']
    unapproved_df.loc[unapproved_df['pbdb_taxon_rank'] == rank, f'{rank}_taxon_id' ] = unapproved_df['pbdb_taxon_id']

log_df(unapproved_df)
# (9024, 39)

(9024, 39)


,taxon_group,verbatim_name,name,genus modifier,genus name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,Any taxon above genus,Any taxon above genus modifier,Comment,Corrections to pbdb_taxon,Notes (change to Internal only notes?),comments,subgenera modifier,subgenera name,species_taxon_id,species_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,?Labyrinthodinium sp. 1,NaN,?,Labyrinthodinium,NaN,sp.,NaN,1,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Dinoflagellates/Acritarchs/Prasinophytes,?Maduradinium sp.,NaN,?,Maduradinium,NaN,sp.,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Dinoflagellates/Acritarchs/Prasinophytes,?Pyxidiella sp. 1,NaN,?,Pyxidiella,NaN,sp.,NaN,1,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,Aandalusiella,NaN,ivoirensis,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,Abratopdinium,NaN,cardioforme,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [35]:
PI_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'Corrections to pbdb_taxon_rank',
       'pbdb_taxon_id.1', 'pbdb_taxon_name.1', 'pbdb_taxon_rank.1',
       'Corrections to pbdb_taxon', 'family_taxon_id', 'family_taxon_name',
       'superfamily_taxon_id', 'superfamily_taxon_name', 'order_taxon_id',
       'order_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id',
       'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [36]:
unapproved_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'genus modifier', 'genus name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name',
       'genus_taxon_id', 'genus_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name', 'subclass_taxon_id', 'subclass_taxon_name',
       'Any taxon above genus', 'Any taxon above genus modifier', 'Comment',
       'Corrections to pbdb_taxon', 'Notes (change to Internal only notes?)',
       'comments', 'subgenera modifier', 'subgenera name', 'species_taxon_id',
       'species_taxon_name'],
      dtype='object')

In [37]:
old_cols = set(unapproved_df.columns)

In [38]:
unapproved_df = unapproved_df.reindex(columns=[
    'taxon_group', 
    'verbatim_name', 'name',
     'Comment', 'Notes (change to Internal only notes?)',
    'Any taxon above genus modifier', 'Any taxon above genus', 
    'genus modifier', 'genus name',
    'subgenera modifier', 'subgenera name',
    'species modifier', 'species name', 
    'subspecies modifier','subspecies name', 
    'non-taxa descriptor', 
    'comments',
    'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 
    'Corrections to pbdb_taxon',
    'species_taxon_id', 'species_taxon_name',
    'genus_taxon_id', 'genus_taxon_name',
    'family_taxon_id', 'family_taxon_name', 
    'order_taxon_id', 'order_taxon_name',
    'subclass_taxon_id', 'subclass_taxon_name',
    'class_taxon_id', 'class_taxon_name', 
    'phylum_taxon_id', 'phylum_taxon_name', 
    'kingdom_taxon_id', 'kingdom_taxon_name',
     'unranked clade_taxon_id', 'unranked clade_taxon_name',   
])
len(unapproved_df.columns)

39

In [39]:
old_cols - set(unapproved_df.columns)

set()

In [40]:
unapproved_df.to_csv(unapproved_taxa_path, index=False)